<a href="https://colab.research.google.com/github/ManuelOtt-code/Project-DC/blob/master/models/With_MF_ML_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import packages
# Install RDKit
!pip install rdkit

# Library imports
from pathlib import Path
from warnings import filterwarnings
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# RDKit
from rdkit import Chem
from rdkit.Chem import rdFingerprintGenerator

# Scikit-learn
from sklearn import svm, metrics, clone
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import (
    auc, accuracy_score, recall_score, roc_curve, roc_auc_score, RocCurveDisplay
)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import (
    VarianceThreshold, SelectKBest, f_classif, mutual_info_classif, chi2
)
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import cohen_kappa_score, confusion_matrix, ConfusionMatrixDisplay





   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.2/35.2 MB 22.3 MB/s eta 0:00:00


ValueError: mount failed

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget --timestamping https://github.com/ManuelOtt-code/Project-DC/raw/refs/heads/master/func.py
!wget --timestamping https://github.com/ManuelOtt-code/Project-DC/raw/refs/heads/master//data_extraction%2Bcuration/Landrum_Phenol_training_data.csv
!wget --timestamping https://github.com/ManuelOtt-code/Project-DC/raw/refs/heads/master//data_extraction%2Bcuration/Landrum_Phenol_test_data.csv
!wget --timestamping https://github.com/ManuelOtt-code/Project-DC/raw/refs/heads/master//data_extraction%2Bcuration/Landrum_All_test_data.csv
!wget --timestamping https://github.com/ManuelOtt-code/Project-DC/raw/refs/heads/master//data_extraction%2Bcuration/Landrum_All_training_data.csv
!wget --timestamping https://github.com/ManuelOtt-code/Project-DC/raw/refs/heads/master//data_extraction%2Bcuration/Minimal_Phenol_training_data.csv
!wget --timestamping https://github.com/ManuelOtt-code/Project-DC/raw/refs/heads/master//data_extraction%2Bcuration/Minimal_Phenol_test_data.csv
!wget --timestamping https://github.com/ManuelOtt-code/Project-DC/raw/refs/heads/master//data_extraction%2Bcuration/Minimal_All_test_data.csv
!wget --timestamping https://github.com/ManuelOtt-code/Project-DC/raw/refs/heads/master//data_extraction%2Bcuration/Minimal_All_training_data.csv


--2025-05-19 23:09:56--  https://github.com/ManuelOtt-code/Project-DC/raw/refs/heads/master/func.py
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ManuelOtt-code/Project-DC/refs/heads/master/func.py [following]
--2025-05-19 23:09:56--  https://raw.githubusercontent.com/ManuelOtt-code/Project-DC/refs/heads/master/func.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6467 (6.3K) [text/plain]
Saving to: ‘func.py’

func.py             100%[===================>]   6.32K  --.-KB/s    in 0s      

Last-modified header missing -- time-stamps turned off.
2025-05-19 23:09:56 (53.6 MB/s) - ‘func.py’ sav

In [ ]:

 #all CSV files -> dataframe
 df_Landrum_Phenol_training_data = pd.read_csv("Landrum_Phenol_training_data.csv")
 df_Landrum_Phenol_test_data = pd.read_csv("Landrum_Phenol_test_data.csv")
 df_Landrum_All_training_data = pd.read_csv("Landrum_All_training_data.csv")
 df_Landrum_All_test_data = pd.read_csv("Landrum_All_test_data.csv")
 df_Minimal_Phenol_training_data = pd.read_csv("Minimal_Phenol_training_data.csv")
 df_Minimal_Phenol_test_data = pd.read_csv("Minimal_Phenol_test_data.csv")
 df_Minimal_All_training_data = pd.read_csv("Minimal_All_training_data.csv")
 df_Minimal_All_test_data = pd.read_csv("Minimal_All_test_data.csv")


In [ ]:
datasets = {
    "Landrum_Phenol_training": df_Landrum_Phenol_training_data,
    "Landrum_All_training": df_Landrum_All_training_data,
    "Landrum_Phenol_test": df_Landrum_Phenol_test_data,
    "Landrum_All_test": df_Landrum_All_test_data,
    "Minimal_Phenol_training": df_Minimal_Phenol_training_data,
    "Minimal_All_training": df_Minimal_All_training_data,
    "Minimal_Phenol_test": df_Minimal_Phenol_test_data,
    "Minimal_All_test": df_Minimal_All_test_data
}

COMPUTE = {
    "Landrum_Phenol_training": False,
    "Landrum_All_training": False,
    "Landrum_Phenol_test": False,
    "Landrum_All_test": False,
    "Minimal_Phenol_training": False,
    "Minimal_All_training": True,
    "Minimal_Phenol_test": False,
    "Minimal_All_test": True
}

# Generate Features



*   all from RDKit
*  all from mordred
*   and add all the the same dataframe







In [ ]:
# prompt: provide a code that generates some features available from rdkit for the structures given in df_curated

def generate_some_rdkit_features(smiles):
    """Generates RDKit features for a given SMILES string."""
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None  # Handle invalid SMILES

    features = {}
    # Descriptors
    features['MolWt'] = Descriptors.MolWt(mol)
    features['LogP'] = Descriptors.MolLogP(mol)
    features['TPSA'] = rdMolDescriptors.CalcTPSA(mol)
    # ... add other RDKit descriptors as needed ...
    # 2D Descriptors
    features['NumHAcceptors'] = Descriptors.NumHAcceptors(mol)
    features['NumHDonors'] = Descriptors.NumHDonors(mol)
    features['NumRotatableBonds'] = Descriptors.NumRotatableBonds(mol)
    features['RingCount'] = Descriptors.RingCount(mol)

    # Topological Descriptors
    features['BalabanJ'] = Descriptors.BalabanJ(mol)
    features['BertzCT'] = Descriptors.BertzCT(mol)
    features['HallKierAlpha'] = Descriptors.HallKierAlpha(mol)


    # ... add other relevant features ...

    return features



In [ ]:
# prompt: provide a code that generates all features available from rdkit for the structures given in df_curated

from rdkit import Chem
from rdkit.Chem.MolStandardize import rdMolStandardize
from rdkit.Chem import Descriptors, rdMolDescriptors

def calculate_rdkit_descriptors_from_mol(smiles):
    """Generates RDKit features for a given SMILES string."""
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None  # Handle invalid SMILES

    features = Descriptors.CalcMolDescriptors(mol)

    return features

def generate_all_rdkit_features(df):
    """
    Calculates all RDKit features for molecules in a DataFrame and adds them as columns.

    Args:
        df: Input DataFrame with a 'canonical_Smiles' column.

    Returns:
        DataFrame with added RDKit features as individual columns.
    """

    # Apply the calculate_rdkit_descriptors_from_mol function to the 'canonical_Smiles' column
    df['features'] = df['canonical_Smiles'].apply(calculate_rdkit_descriptors_from_mol)

    # Expand the features dictionary into separate columns
    features_df = pd.DataFrame(df['features'].tolist(), index=df.index)  # Use index of original df

    # Concatenate the expanded features with the original DataFrame
    df = pd.concat([df, features_df], axis=1)

    # Drop the original features column
    df = df.drop('features', axis=1)

    return df

In [ ]:
# prompt: provide a function to calculate all 2D descriptors from mordred using the smiles from the df dataframe and append the calculated features to the same dataframe



def generate_mordred_descriptors(df):
    """Calculates all 2D descriptors from Mordred and appends them to the DataFrame.

    Args:
        df: Input DataFrame with a 'canonical_Smiles' column.

    Returns:
        DataFrame with added Mordred descriptors.
    """

    calc = Calculator(descriptors, ignore_3D=True) # Initialize Mordred calculator (2D only)

    # Calculate descriptors for valid molecules
    def calculate_mordred_descriptors_for_mol(smiles):
      mol = Chem.MolFromSmiles(smiles)
      if mol is not None:
          return calc(mol)
      else:
          return None  # Return None for invalid SMILES

    df['mordred_descriptors'] = df['canonical_Smiles'].apply(calculate_mordred_descriptors_for_mol)

    # Expand the Mordred descriptor dictionary into separate columns
    mordred_df = pd.DataFrame(df['mordred_descriptors'].tolist())

    # Concatenate the expanded features with the original DataFrame
    df = pd.concat([df, mordred_df], axis=1)

    # Drop the original mordred_descriptors column
    df = df.drop('mordred_descriptors', axis=1)

    return df




In [ ]:
#Generate rdkit features

rdkit_features = {}

for name, df in datasets.items():
    if COMPUTE.get(name, False):
        print(f"✅ Computing: {name}")
        rdkit_features[name] = generate_all_rdkit_features(df)
        rdkit_features[name].to_csv(f"{name}_rdkit.csv", index=False)
    else:
        print(f"⏭️ Skipping: {name}")







⏭️ Skipping: Landrum_Phenol_training
⏭️ Skipping: Landrum_All_training
⏭️ Skipping: Landrum_Phenol_test
⏭️ Skipping: Landrum_All_test
⏭️ Skipping: Minimal_Phenol_training
✅ Computing: Minimal_All_training
⏭️ Skipping: Minimal_Phenol_test
✅ Computing: Minimal_All_test


In [ ]:
!pip install mordred
!pip install numpy==1.23.5
from mordred import Calculator, descriptors


In [ ]:
#Generate mordred features
mordred_descriptors = {}

for name, df in datasets.items():
    if COMPUTE.get(name, False):
        print(f"✅ Computing: {name}")
        mordred_descriptors[name] = generate_mordred_descriptors(df)
        mordred_descriptors[name].to_csv(f"{name}_mordred.csv", index=False)
    else:
        print(f"⏭️ Skipping: {name}")




⏭️ Skipping: Landrum_Phenol_training
⏭️ Skipping: Landrum_All_training
⏭️ Skipping: Landrum_Phenol_test
⏭️ Skipping: Landrum_All_test
⏭️ Skipping: Minimal_Phenol_training
✅ Computing: Minimal_All_training


/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


⏭️ Skipping: Minimal_Phenol_test
✅ Computing: Minimal_All_test


/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


#Feature selection

* Remove semi-constant features (>= 80% of column values the same after
* Lu, A. (2022) https://doi.org/10.1038/s41598-022-11925-y)
Remove highly correlating columns. Threshold to be discussed, maybe 0.75?

In [ ]:
#alternative feature selection

def drop_non_numeric_columns(df):
    """Drops columns from a DataFrame that do not contain numeric values.

    Args:
        df: The input DataFrame.

    Returns:
        A new DataFrame with non-numeric columns removed.
    """
    numeric_df = df.select_dtypes(include=np.number)
    return numeric_df

In [ ]:
#old feature selectionfunctions (probably delete)
def remove_semi_constant_features(df, semi_constant_threshold):
    """Removes columns from a Pandas DataFrame where 80% or more of the values are the same.

    Args:
        df: The input DataFrame.

    Returns:
        A new DataFrame with the single-value columns removed.
    """

    cols_to_drop = []
    for col in df.columns:
        try:
            # Calculate the percentage of the most frequent value
            counts = df[col].value_counts(normalize=True)
            if counts.iloc[0] >= semi_constant_threshold:
                cols_to_drop.append(col)
        except (TypeError, IndexError):
            pass  # Handle cases where value_counts fails (e.g., mixed data types)

    return df.drop(columns=cols_to_drop)

def drop_non_numeric_columns(df):
    """Drops columns from a DataFrame that do not contain numeric values.

    Args:
        df: The input DataFrame.

    Returns:
        A new DataFrame with non-numeric columns removed.
    """
    numeric_df = df.select_dtypes(include=np.number)
    return numeric_df
def drop_high_correlation_columns(df, threshold=0.75):
    """Drops columns from a DataFrame that have a correlation above a specified threshold.

    Args:
        df: The input DataFrame.
        threshold: The correlation threshold above which columns are dropped.

    Returns:
        A new DataFrame with highly correlated columns removed.
    """

    # Calculate the correlation matrix
    corr_matrix = df.corr().abs()

    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

    # Find index of feature columns with correlation greater than 0.75
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

    # Drop features
    return df.drop(columns=to_drop)






In [ ]:

Landrum = True
Phenol = False

if Landrum:
    if Phenol:
        df_training = pd.read_csv("Landrum_Phenol_training_mordred.csv")
        df_test = pd.read_csv("Landrum_Phenol_test_mordred.csv")
    else:
        df_training = pd.read_csv("Landrum_All_training_mordred.csv")
        df_test = pd.read_csv("Landrum_All_test_mordred.csv")
else:
    if Phenol:
        df_training = pd.read_csv("Minimal_Phenol_training_mordred.csv")
        df_test = pd.read_csv("Minimal_Phenol_test_mordred.csv")
    else:
        df_training = pd.read_csv("Minimal_All_training_mordred.csv")
        df_test = pd.read_csv("Minimal_All_test_mordred.csv")

# === Normalize column names just once ===
df_training.columns = df_training.columns.str.strip().str.lower()
df_test.columns = df_test.columns.str.strip().str.lower()

# Configuration switches
USE_MORDRED = True       # True for Mordred, False for Morgan

if USE_MORDRED:
    # Drop non-numeric columns and NaNs
    df_training = drop_non_numeric_columns(df_training)
    df_training = df_training.dropna(axis=1)
    df_test = drop_non_numeric_columns(df_test)
    df_test = df_test.dropna(axis=1)

    # Prepare features and labels
    X_train = df_training.drop(columns=["active", "is_phenol"], errors='ignore')
    y_train = df_training["active"]
    X_test = df_test.drop(columns=["active", "is_phenol"], errors='ignore')
    y_test = df_test["active"]


else:
    # Process Morgan fingerprints from SMILES
    mfpgen = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=2048)

    def smiles_to_fp(smiles):
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return np.nan
        return mfpgen.GetFingerprintAsNumPy(mol)

    df_training["fp"] = df_training["canonical_smiles"].apply(smiles_to_fp)
    df_training = df_training[df_training["fp"].notna()].copy()
    df_test["fp"] = df_test["canonical_smiles"].apply(smiles_to_fp)
    df_test = df_test[df_test["fp"].notna()].copy()


    X_training = pd.DataFrame(df_training["fp"].to_list())
    y_training = df_training["active"]
    X_test = pd.DataFrame(df_test["fp"].to_list())
    y_test = df_test["active"]





<ipython-input-22-bd99857b4b78>:9: DtypeWarning: Columns (7,8,9,10,11,12,13,14,15,16,17,18,56,57,58,59,60,61,62,63,64,143,144,152,153,155,156,157,158,159,160,161,170,171,179,180,188,189,197,198,206,207,215,216,224,225,233,234,236,237,238,239,240,241,242,243,244,263,264,265,266,267,268,269,270,271,344,345,346,347,348,349,350,359,360,368,369,371,372,373,374,375,376,377,386,387,395,396,404,405,413,414,422,423,431,432,440,441,449,450,452,453,454,455,456,457,465,466,473,474,476,477,478,479,480,481,489,490,497,498,505,506,513,514,521,522,529,530,537,538,545,546,548,549,550,551,552,553,561,562,569,570,572,573,574,575,576,577,585,586,593,594,601,602,609,610,617,618,625,626,633,634,641,642,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,72

In [ ]:
X_train_np = X_train.to_numpy()
y_train_np = y_train.to_numpy()
X_test_np = X_test.to_numpy()
y_test_np = y_test.to_numpy()

print(X_train_np[:5])

[[ 13.23443095  11.54854293   1.         ...  82.          88.
    4.11111111]
 [ 31.0649762   21.62099313   0.         ... 220.         272.
    8.        ]
 [ 17.2868016   14.56386614   0.         ... 114.         131.
    5.25      ]
 [ 23.84538077  19.03224991   0.         ... 158.         183.
    6.83333333]
 [ 16.33240565  13.45331614   0.         ... 106.         120.
    5.19444444]]


#Model training

In [ ]:
# Define Preprocessing and Models
from sklearn.metrics import classification_report

def get_model_pipeline(model_name):
    if model_name == 'RandomForest':
        model = RandomForestClassifier(class_weight='balanced', random_state=42)
    elif model_name == 'SVM':
        model = SVC(class_weight='balanced', probability=True, random_state=42)
    elif model_name == 'LogisticRegression':
        model = LogisticRegression(class_weight='balanced', solver='liblinear', random_state=42)
    elif model_name == 'NeuralNetwork':
        model = MLPClassifier(max_iter=1000, random_state=42)
    else:
        raise ValueError(f"Model {model_name} is not supported.")

    pipeline = Pipeline([
        ('scaler', MinMaxScaler()),
        ("var_thresh", VarianceThreshold(threshold=0.01)),#light filtering of low variance features (more often seen than the 80% constant value method but should do approx the same)
        ("anova", SelectKBest(score_func=mutual_info_classif)),  # Supervised Filter-Based Selection, k can be tuned (chi2 for morgan, f_classif for mordred, mutual_info_classif can handle both but takes long)
        ('classifier', model)
    ])

    return pipeline

In [ ]:
param_grids = {
    'RandomForest': {
        'classifier__n_estimators': [100, 200],
        'classifier__max_depth': [None, 10, 20],
        'anova__k': [50, 100, 200],
        'classifier__max_features': ['sqrt', 'log2']
    },
    'SVM': {
        'classifier__C': [0.1, 1, 10],
        'anova__k': [50, 100, 200],
        'classifier__kernel': ['linear', 'rbf']
    },
    'LogisticRegression': {
    'classifier__penalty': ['l1', 'l2', 'elasticnet'],
    'classifier__C': [0.1, 0.01, 0.001],
    'classifier__solver': ['saga'],
    'anova__k': [50, 100, 200],
    'classifier__l1_ratio': [0.25, 0.5, 0.75]
    },
    'NeuralNetwork': {
        'classifier__hidden_layer_sizes': [(50,), (100,), (50, 50)],
        'classifier__alpha': [0.0001, 0.001],
        'anova__k': [50, 100, 200],
        'classifier__learning_rate': ['constant', 'adaptive']
    }
}


In [ ]:
#Hyperparameter Tuning with Cross-Validation
from sklearn.model_selection import GridSearchCV, StratifiedKFold
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
def perform_grid_search(pipeline, param_grid, X_train, y_train):
    grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        cv=kf,
        scoring='roc_auc',
        n_jobs=-1
    )
    grid_search.fit(X_train, y_train)
    return grid_search


In [ ]:
#Model Evaluation
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    accuracy_score, roc_auc_score, precision_score, f1_score, roc_curve
)
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from sklearn.base import clone

#everything called test has nothing to do with test set (is validation but called test in scikit learn)
def evaluate_model_with_cv(model, X, y, random_state):
    """
    Evaluate a model using 10-fold cross-validation and return performance metrics.

    Parameters:
        model: The model to evaluate
        X: Feature matrix
        y: Target vector
        random_state: Random seed for reproducibility

    Returns:
        Dictionary with performance metrics and arrays of per-fold metrics
    """
    t0 = time.time()
    # Initialize KFold
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=random_state)


    # Lists to store metrics
    metrics = {
        'accuracies': [],
        'precisions': [],
        'f1_scores': [],
        'auc_scores': [],
        'kappa_scores': []
    }

    # Create figure for ROC curves
    plt.figure()

    # Iterate over folds
    for k, (train_index, test_index) in enumerate(skf.split(X, y)):
        # Split data
        X_train_fold, X_test_fold = X[train_index], X[test_index]
        y_train_fold, y_test_fold = y[train_index], y[test_index]

        # Train model
        model.fit(X_train_fold, y_train_fold)

        # Predict
        y_pred_fold = model.predict(X_test_fold)
        y_prob_fold = model.predict_proba(X_test_fold)[:, 1]

        # Calculate metrics
        metrics['accuracies'].append(accuracy_score(y_test_fold, y_pred_fold))
        metrics['precisions'].append(precision_score(y_test_fold, y_pred_fold))
        metrics['f1_scores'].append(f1_score(y_test_fold, y_pred_fold))
        metrics['auc_scores'].append(roc_auc_score(y_test_fold, y_prob_fold))
        metrics['kappa_scores'].append(cohen_kappa_score(y_test_fold, y_pred_fold))

        # ROC curve
        RocCurveDisplay.from_predictions(y_test_fold, y_prob_fold, name=f"Fold {k+1}", ax=plt.gca())


    # Print metrics
    metric_display_names = {
    'accuracies': 'accuracy',
    'precisions':'precision',
    'f1_scores': 'F1 score',
    'auc_scores': 'AUC score',
    'kappa_scores': 'Cohens kappa'
    }

    for metric_name, values in metrics.items():
      display_name = metric_display_names[metric_name]
      print(f"Average {display_name}: {np.mean(values):.4f} \t and std err: {stats.sem(values):.4f}")
    print(f"Time taken : {time.time() - t0:.2f}s\n")
    return metrics

In [ ]:
# loop through each model, perform grid search, and evaluate
import matplotlib.pyplot as plt
from scipy.stats import sem
for model_name in ['RandomForest']: #add models to train: 'NeuralNetwork','RandomForest', 'SVM', 'LogisticRegression',
    print(f"\nTraining and evaluating {model_name}...")
    t0 = time.time()
    pipeline = get_model_pipeline(model_name)
    param_grid = param_grids[model_name]
    grid_search = perform_grid_search(pipeline, param_grid, X_train, y_train)
    model = grid_search.best_estimator_
    print(f"Time taken : {time.time() - t0:.2f}s\n")
    print(f"Hyperparameters: {grid_search.best_params_}\n")
    evaluate_model_with_cv(model, X_train, y_train, random_state=22)


Training and evaluating NeuralNetwork...
